<a href="https://colab.research.google.com/github/ahmer-talal/Neural-Networks/blob/main/Regression_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch numpy scikit-learn


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


## **Creating Dataset:** for Regression's practice using PyTorch

In [3]:
# 1000 samples, 5 features
X, y = make_regression(n_samples=1000, n_features=5, noise=20, random_state=42)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale (important for NN)
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test = scaler_y.transform(y_test.reshape(-1, 1))

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


## **Defining Model**

In [4]:
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

model = RegressionModel(input_dim=5)


## **Loss & Optimizer**

In [5]:
criterion = nn.MSELoss()        # Regression's common loss
optimizer = optim.Adam(model.parameters(), lr=0.01)


## **Training Loop**

In [6]:
epochs = 100

for epoch in range(epochs):
    model.train()

    optimizer.zero_grad()         # old gradients clear
    outputs = model(X_train)      # forward pass
    loss = criterion(outputs, y_train)
    loss.backward()               # backward pass
    optimizer.step()              # weights update

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [10/100], Loss: 0.1869
Epoch [20/100], Loss: 0.1340
Epoch [30/100], Loss: 0.1061
Epoch [40/100], Loss: 0.0948
Epoch [50/100], Loss: 0.0916
Epoch [60/100], Loss: 0.0888
Epoch [70/100], Loss: 0.0870
Epoch [80/100], Loss: 0.0859
Epoch [90/100], Loss: 0.0849
Epoch [100/100], Loss: 0.0841


## **Testing / Evaluation**

In [7]:
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    test_loss = criterion(predictions, y_test)
    print(f"Test Loss: {test_loss.item():.4f}")


Test Loss: 0.1036
